# CodeMind fine tuning
## Methods:
* model: gemma-2b-it

In [ ]:
from google.colab import userdata, drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/CodeMind/

!pip install -r requirements.txt --upgrade

In [2]:
import datasets
import numpy as np
import torch
import evaluate
from dotenv import load_dotenv
from transformers import Trainer, AutoTokenizer, AutoModelForCausalLM, TrainingArguments, default_data_collator

load_dotenv()

False

In [3]:
import wandb

wandb.login()

wandb.init(
    project='llama2-7b-chat',
    name='full fine-tuning',
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jehwan-kim (codemind). Use `wandb login --relogin` to force relogin


In [4]:
# from huggingface_hub import notebook_login

# notebook_login()

In [5]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'
token = userdata.get('HF_READ')

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          trust_remote_code=True,
                                          token=token)
tokenizer.pad_token = tokenizer.convert_ids_to_tokens(0)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    trust_remote_code=True,
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
df = datasets.load_dataset('kreimben/leetcode_user_submissions', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/109309 [00:00<?, ? examples/s]

In [10]:
def formatting_func(d):
    return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: You are kind teacher which teaches coding test questions like leetcode. Please provide step by step instruction and insight for users.

### Input: I don't know how to approach {d['title_slug']} problem.

### Response: {d['content']}
""".strip()

#  {d['question_content']}\nTag: {d['tag']}\nLevel: {d['level']}\nHints: {d['question_hints']}


def tokenize(example):
    full_prompt = formatting_func(example)
    tokenized = tokenizer(full_prompt, padding='max_length', truncation=True, max_length=512)
    return tokenized


df = df.map(tokenize)
df = df.remove_columns(
    ['question_content', 'title_slug', 'tag', 'level', 'question_hints', 'view_count', 'vote_count', 'content'])
train_dataset, test_dataset = df.train_test_split(test_size=0.2, seed=42).values()
train_dataset, test_dataset

Map:   0%|          | 0/109309 [00:00<?, ? examples/s]

OverflowError: int too big to convert

In [ ]:
metric_bleu = evaluate.load("bleu")
metric_rouge = evaluate.load("rouge")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute BLEU score
    bleu_score = metric_bleu.compute(predictions=[decoded_preds], references=[decoded_labels])

    # Compute ROUGE scores
    rouge_scores = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "bleu": bleu_score["bleu"],
        "rouge1": rouge_scores["rouge1"],
        "rouge2": rouge_scores["rouge2"],
        "rougeL": rouge_scores["rougeL"],
    }

In [ ]:
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

In [13]:
ta = TrainingArguments(
    output_dir='out',
    overwrite_output_dir=True,

    fp16=True,
    # optim="paged_adamw_8bit",
    lr_scheduler_type='cosine',
    warmup_ratio=.05,

    logging_steps=1000,
    report_to='wandb',

    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,

    # eval_steps=1000,
    num_train_epochs=3,

    do_train=True,
    do_eval=True,

    metric_for_best_model="train_loss",
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=ta,
    compute_metrics=compute_metrics,
    data_collator=default_data_collator,
    callbacks=[early_stopping],
)

In [ ]:
trainer.train()

In [ ]:
# upload the trained model to huggingface.
peft_model_id = 'kreimben/CodeMind'
write_token = userdata.get('HF_WRITE')
revision_id = 'llama2-7b-chat-20240408'

In [ ]:
model.push_to_hub(
    repo_id=peft_model_id,
    token=write_token,
    revision=revision_id,
)

# generation_config = GenerationConfig(
#     penalty_alpha=.6,
#     do_sample=True,
#     top_k=5,
#     temperature=.5,
#     repetition_penalty=1.2,
#     max_new_tokens=512,
#     pad_token_id=tokenizer.eos_token_id,
# )

# generation_config.push_to_hub(
#     repo_id=peft_model_id,
#     token=write_token,
#     revision=revision_id,
# )

In [ ]:
del trainer
del data_collator
del ta
del train_dataset
del test_dataset
del model
del tokenizer
del model_id
del metric_bleu
del metric_rouge

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained(peft_model_id, revision=revision_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id, revision=revision_id, device_map='cuda', load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

In [ ]:
# test the model
from transformers import pipeline

system_prompt = f"""
"""


def formatting_func(user_input):
    global system_prompt
    return f"<human>: {system_prompt}\n\n{user_input}\n<assistant>: "


def generate_response(user_input):
    prompt = formatting_func(user_input)

    pipe = pipeline(
        'text-generation',
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
    )

    return pipe(prompt)


In [ ]:
%%time

with torch.no_grad():
    res = generate_response('leetcode number 1 add sum')

# model.generation_config

In [ ]:
res